In [1]:

using ClusterMeanField
using PyCall
using LinearAlgebra
using Printf
using InCoreIntegrals
using QCBase
using ActiveSpaceSolvers
using RDM

In [2]:
molecule = "
He         0.52436       -0.90821        0.75187
He         0.52436        0.90821        0.75187
He        -1.04871        0.00000        0.75187
He        -1.04871        0.00000       -0.75187
He         0.52436        0.90821       -0.75187
He         0.52436       -0.90821       -0.75187
He        -0.00000       -0.00000        2.23447
He        -0.00000       -0.00000       -2.23447
"
atoms = []
for (li,line) in enumerate(split(rstrip(lstrip(molecule)), "\n"))
    l = split(line)
    push!(atoms, Atom(li, l[1], parse.(Float64,l[2:4])))
end

#basis = "aug-cc-pvdz" #9 orbs on each He
basis = "cc-pvdz" #5 orbs on each He
#basis = "cc-pvtz" # 6 orbs on each He
#basis = "sto-3g"

# Create FermiCG.Molecule type
mol = Molecule(0,1,atoms,basis)

pyscf = pyimport("pyscf")
pymol = pyscf.gto.Mole(atom=molecule, spin=0, charge=0, basis=basis)
pymol.build()
mf = pyscf.scf.RHF(pymol).run()
s = mf.get_ovlp(pymol)

lo = pyimport("pyscf.lo.orth")
lo_ao = lo.lowdin(s)
println("size of Lowdin ortho AO's:", size(lo_ao))

ClusterMeanField.pyscf_write_molden(mol, lo_ao, filename="lowdin_ao_augccpvdz_he08_07.molden")

#write fci dump file from the modified mo coefficients
tools = pyimport("pyscf.tools")
tools.fcidump.from_mo(pymol, "fcidump.he08_07", lo_ao)

converged SCF energy = -22.7349718943626
size of Lowdin ortho AO's:(40, 40)


/Users/ayush/workspace/cmf/ClusterMeanField.jl/cmf_env/lib/python3.10/site-packages/pyscf/lib/misc.py:141: UserWarning: OpenMP is not available. Setting omp_threads to 1 has no effects.
  warnings.warn('OpenMP is not available. '
/Users/ayush/workspace/cmf/ClusterMeanField.jl/cmf_env/lib/python3.10/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [3]:
#Can just read in pyscf dump file for integrals (once you have already run an scf calculation)
pyscf = pyimport("pyscf");
fcidump = pyimport("pyscf.tools.fcidump");
ctx = fcidump.read("fcidump.he08_07");
h = ctx["H1"];
g = ctx["H2"];
ecore = ctx["ECORE"];
g = pyscf.ao2mo.restore("1", g, size(h,2))

ints = InCoreInts(ecore,h,g);

np=pyimport("numpy")
np.save("ints_h0_he08_07",ints.h0)
np.save("ints_h1_he08_07",ints.h1)
np.save("ints_h2_he08_07",ints.h2)